In [7]:
!python -V
import torch
import PIL
import PIL.Image
import numpy as np
from rknn.api import RKNN
model = torch.load("./resnet18_imagenette_model/resnet18.pt")
model.eval()

dummy_input = torch.randn(1, 3, 224, 224)
torch.onnx.export(model, dummy_input, "./resnet18_imagenette_model/resnet18.onnx", opset_version=11)

Python 3.8.20


In [8]:
rknn_runtime = RKNN(verbose=True, verbose_file="./resnet_build.log")
# normalization of Imagenette
# dataset_normalization = Normalize(mean=[0.4648, 0.4543, 0.4247], std=[0.2785, 0.2735, 0.2944])
mean=[0.4648, 0.4543, 0.4247]
std=[0.2785, 0.2735, 0.2944]

rknn_runtime.config(
    mean_values=[[v * 255. for v in mean]],
    std_values=[[v * 255. for v in std]],
    quant_img_RGB2BGR=True,
    target_platform="RV1106",
    quantized_algorithm="normal",
    quantized_method="channel",
    optimization_level=3,
    quantized_dtype="asymmetric_quantized-8",
    custom_string="saul_v0.0.1",
)

ret = rknn_runtime.load_onnx(model="./resnet18_imagenette_model/resnet18.onnx")
if ret != 0:
    raise TypeError(f"model load error, {ret}")

ret = rknn_runtime.build(do_quantization=True, dataset="./dataset.txt")
if ret != 0:
    raise TypeError(f"rknn build error, ret: {ret}")

ret = rknn_runtime.export_rknn('./resnet18_imagenette_model/resnet18.rknn')
if ret !=0 :
    raise TypeError(f"export RKNN model error, ret: {ret}")



I Save log info to: ./resnet_build.log
W __init__: rknn-toolkit2 version: 1.5.2+b642f30c
W config: The quant_img_RGB2BGR of input 0 is set to True, which means that the RGB2BGR conversion will be done first
          when the quantized image is loaded (only valid for jpg/jpeg/png/bmp, npy will ignore this flag).
          Special note here, if quant_img_RGB2BGR is True and the quantized image is jpg/jpeg/png/bmp,
          the mean_values / std_values in the config corresponds the order of BGR.
W load_onnx: It is recommended onnx opset 12, but your onnx model opset is 11!
W load_onnx: Model converted from pytorch, 'opset_version' should be set 12 in torch.onnx.export for successful convert!
             More details can be found in examples/pytorch/resnet18_export_onnx
Loading : 100%|███████████████████████████████████████████████████| 42/42 [00:00<00:00, 3889.45it/s]
I base_optimize ...
I base_optimize done.
I 
I fold_constant ...
I fold_constant done.
I 
I correct_ops ...
I correct_o

I RKNN: [10:25:35.007] compress = 0, conv_eltwise_activation_fuse = 1, global_fuse = 1, multi-core-model-mode = 7, output_optimize = 1,enable_argb_group=0
I RKNN: librknnc version: 1.5.2 (c6b7b351a@2023-08-23T15:34:44)
D RKNN: [10:25:35.038] RKNN is invoked
D RKNN: [10:25:35.108] >>>>>> start: N4rknn19RKNNSetOpTargetPassE
D RKNN: [10:25:35.108] <<<<<<<< end: N4rknn19RKNNSetOpTargetPassE
D RKNN: [10:25:35.108] >>>>>> start: N4rknn16RKNNAddFirstConvE
D RKNN: [10:25:35.108] <<<<<<<< end: N4rknn16RKNNAddFirstConvE
D RKNN: [10:25:35.108] >>>>>> start: N4rknn27RKNNEliminateQATDataConvertE
D RKNN: [10:25:35.108] <<<<<<<< end: N4rknn27RKNNEliminateQATDataConvertE
D RKNN: [10:25:35.108] >>>>>> start: N4rknn17RKNNTileGroupConvE
D RKNN: [10:25:35.108] <<<<<<<< end: N4rknn17RKNNTileGroupConvE
D RKNN: [10:25:35.108] >>>>>> start: N4rknn19RKNNTileFcBatchFuseE
D RKNN: [10:25:35.108] <<<<<<<< end: N4rknn19RKNNTileFcBatchFuseE
D RKNN: [10:25:35.108] >>>>>> start: N4rknn15RKNNAddConvBiasE
D RKNN: [10:25

In [9]:
ret = rknn_runtime.init_runtime(target=None)
if ret != 0:
    raise Exception(f"rknn init fail: {ret}")


img = PIL.Image.open("dataset/imagenette2-320/train/n02102040/ILSVRC2012_val_00000665.JPEG").resize((224, 224))
x_np = np.array(img)
x_np = np.expand_dims(x_np, axis=0)
# x_tc = torch.from_numpy(x_np).permute(2, 0 ,1).unsqueeze(0)

outputs = rknn_runtime.inference(inputs=[x_np])
print(outputs[0].shape)
y = np.argmax(outputs[0])
print(y)

W init_runtime: Target is None, use simulator!
W inference: The 'data_format' has not been set and defaults is nhwc!
Preparing : 100%|██████████████████████████████████████████████████| 51/51 [00:00<00:00, 162.58it/s]


(1, 10)
1


In [10]:
rknn_runtime.release()